In [ ]:
'''
This script simulates the application in Section 5 of "Principles, evaluation metrics and method for planned regulatory inspection targeting".
'''
__author__ = 'Celso Ribas'
__credits__ = ['Celso Ribas', 'José Bermudez']
__version__ = '0.1'
__maintainer__ = 'Celso Ribas'
__contact__ = 'celsoribas@gmail.com'
__status__ = 'dev'
__date__ = 'November 1st, 2022'

In [ ]:
# loading libraries
import numpy as np
from scipy.sparse import bsr_matrix
import tabulate
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse
import time

In [ ]:
# inputs for Monte-Carlo simulation
# SET THESE VARIABLES PROPERLY
realizations = 10  # number of realizations to be simulated
rounds = 1600  # number of inspection rounds to be simulated in a realization
Mu = np.array([5., 1., 0.5])  # three average capacity of one inspector at a standard municipality 
R = 224  # maximum quantities of inspectors acting in an inspection round
M = 56  # maximum number of municipalities chosen in an inspection round
mu_eta = 0  # mean of noise in consumer complaint rate
sigma_eta = 0.5  # std of noise in consumer complaint rate
delta = np.array([5, 2, 1])  # deltas for changes (Figure 5)
mu_ch = Mu[1]  # mu for simulate changes (Figures 6 and 7)
delta_ch = delta[0] # # delta for simulate changes (Figures 6 and 7)
N_ch = 56 # number of municipalities where the change takes place (Figures 6 and 7)
n_ch = 100 # inspection round when the change takes place (Figures 6 and 7)

In [ ]:
# functions

def PRInT(tau):
    '''
    Allocate inspectors according to the number of consumers in each municipality, using as a basis one inspector for every 50, 000 consumers.

    Parameters
    ----------
    tau : (M, ) ndarray
        municipalities where to allocate inspectors.
    
    Returns
    -------
    tau : (?, ) ndarray
        municipalities where inspectors were effectively allocated.
    q_t : (?, ) ndarray
        quantities of inspectors allocated to the municipalities in tau
        
    '''
    global insp_cons, R, M  # these parameters are fixed
    cum_insp = np.cumsum(insp_cons[tau], dtype=int)  # accumulated inspectors needed for chosen municipalities
    mult_factor = R // cum_insp[-1]  # how many times R is greater than necessary
    q_t = np.zeros(M, dtype=int)  # array for the quantities of inspectors allocated for the chosen municipalities 
    if mult_factor > 0:
        q_t = mult_factor * insp_cons[tau]  # quantities of inspectors allocated for the chosen municipalities
    insp_remain = R % cum_insp[-1]  # quantities of inspectors remaining to be allocated
    if insp_remain > 0:
        vt_remain = (cum_insp > insp_remain).argmax().astype(int)  # number of municipalities to be targeted with the inspectors remaining
        q_t[:vt_remain] = q_t[:vt_remain] + insp_cons[tau[:vt_remain]]  # adding the inspectors remaining to vt_remain
        diff = R - q_t.sum()  # checking if more than M inspectors were allocated to the last municipality
        q_t[vt_remain] = q_t[vt_remain] + diff  # removing the excess from the vt_remain
    q_t = q_t[q_t != 0]  # dropping municipalities with zero allocated inspectors 
    tau = tau[:q_t.shape[0]]  # dropping municipalities initially chosen but which will not receive inspectors
    return tau, q_t  # municipalities chosen and quantities of inspectors allocated

def PRInT_rt(x_i):
    '''
    Allocate inspectors to municipalities with the higher consumer complaint rates.
    The quantities of inpectors are determined according to the number of consumers in each municipality, using as a basis one inspector for every 50, 000 consumers.

    Parameters
    ----------
    x_i : (vertices, ) ndarray
        consumer complaint rates of municipalities.
    
    Returns
    -------
    tau : (?, ) ndarray
        municipalities where inspectors were effectively allocated.
    q_t : (?, ) ndarray
        quantities of inspectors allocated to the municipalities in tau
        
    '''
    global insp_cons, R, M  # these parameters are fixed
    tau_temp = np.argpartition(x_i, -M)[-M:]  # municipalities with the M higher consumer complaint rates
    tau_ord = x_i[tau_temp].argsort()[::-1]  # order within the M higher consumer complaint rates
    tau = tau_temp[tau_ord]  # sorted municipalities with of M higher consumer complaint rates
    tau, q_t = PRInT(tau)  # municipalities chosen and quantity of inspectors allocated
    return tau, q_t

def resp_of_pr(z_i, y_i, f, th, n):
    '''
    Flag municipalities whether the changes will be detected by proposed method

    Parameters
    ----------
    z_i : (vertices, ) ndarray
        consumer complaint rates of municipalities.
    y_i : (vertices, ) ndarray
        relevant regulator's expectations.
    f : float
        value for function f in line 9 of algorithm 1.
    th : float
        threshold value of the criteria used by proposed method
    n : int
        inspection round        
    '''    
    global delta, ch_pr_a, ch_pr_b, ch_pr_c, mu_i, realization # these parameters are fixed
    
    # change a 
    d_i_a = (z_i + delta[0])/y_i # relevant discrepancy if consumer complaint rates increase delta[0]
    r_i_a = z_i + delta[0] - mu_i # new consumer complaint rates after possible reduction
    mask_r_a = r_i_a < 0 # mask for new consumer complaint rates after possible reduction
    r_i_a[mask_r_a] = 0 # limiting new consumer complaint rates after possible reduction to non-negative values
    p_i_a = r_i_a/y_i # relevant discrepancy of municipalities after possible reduction
    mask_th_a = d_i_a - f*p_i_a > th # mask for municipalities whether the change will be detected
    ch_pr_a[mask_th_a, n, realization] = 1 # flag municipalities whether the change will be detected
    
    # change b 
    # the same as change a but with delta[1]
    d_i_b = (z_i + delta[1])/y_i
    r_i_b = z_i + delta[1] - mu_i
    mask_r_b = r_i_b < 0
    r_i_b[mask_r_b] = 0
    p_i_b = r_i_b/y_i
    mask_th_b = d_i_b - f*p_i_b > th
    ch_pr_b[mask_th_b, n, realization] = 1
    
    # change c 
    # the same as change a but with delta[2]
    d_i_c = (z_i + delta[2])/y_i
    r_i_c = z_i + delta[2] - mu_i
    mask_r_c = r_i_c < 0
    r_i_c[mask_r_c] = 0
    p_i_c = r_i_c/y_i
    mask_th_c = d_i_c-f*p_i_c > th
    ch_pr_c[mask_th_c, n, realization] = 1

def PRInT_pr(z, f, n, resp=True):
    '''
    Allocate inspectors to municipalities according the proposed method

    Parameters
    ----------
    z : (vertices, ) ndarray
        consumer complaint rates of municipalities.
    f : float
        value of function f in line 9 of algorithm 1.
    n : integer
        value of inspection round.
    resp : bool
        value to indicate whether the method's responsiveness should be calculated
        
    Returns
    -------
    tau : (?, ) ndarray
        municipalities where inspectors were effectively allocated.
    q_t : (?, ) ndarray
        quantities of inspectors allocated to the municipalities in tau
        
    '''
    global R, Ws, mu_i, M, W # these parameters are fixed
    z_i = z.copy()  # necessary for not to change the value of z
    m = 0  # number of already chosen municipalities 
    r = 0  # quantity of already allocated inspectors
    tau = np.array([], dtype=int)  # array for the chosen municipalities
    q_t = np.zeros((z_i.size, ), dtype=int)  # array for the quantities of inspectors allocated for the chosen municipalities
    mask_vt = np.zeros((z_i.size, ), dtype=bool)  # mask for municipalities that cannot be chosen
    y_i = Ws.dot(z_i)  # regulator's expectations for consumer complaint rates
    mask_y = y_i < 1  # mask for the lower bound of regulator's expectations
    y_i[mask_y] = 1  # relevant regulator's expectations
    d_i = z_i/y_i  # relevant discrepancy of municipalities
    for r in range(R):  # for each inspector
        r_i = z_i - mu_i  # consumer complaint rates after possible reductions
        mask_r = r_i < 0  # mask for consumer complaint rates after possible reductions
        r_i[mask_r] = 0  # limiting consumer complaint rates to non-negative values
        p_i = r_i/y_i  # relevant discrepancy of municipalities after possible reductions
        d_i[mask_vt] = -1  # forcing the criteria to a negative value if the municipality does not belong to possible chosen municipalities
        red = d_i-f*p_i  # reduction in the relevant discrepancy after possible reductions on complaint rates
        v_t = np.argmax(red).astype(int) # finding the municipality with largest weighted reduction in the relevant discrepancy
        th = np.max(red) # the largest weighted reduction in the relevant discrepancy is the threshold for checking whether a change will be detected
        if resp:
            if m < M: # if the number of municipalities chosen has reached the maximum, a change in the complaints rate will not cause a new municipality to be chosen
                resp_of_pr(z_i, y_i, f, th, n) # municipalities whether the changes will be detected 
        if v_t not in tau:  # if the chosen municipality was not chosen yet
            tau = np.append(tau, v_t)  # include the chosen municipality in the set of chosen municipalities
            m += 1  # add 1 to the number of chosen municipalities
            if m == M:  # if M was already achived
                mask_vt = np.ones((z_i.size, ), dtype=bool)  # set that no municipality can be chosen
                mask_vt[tau] = False  # set that just the already chosen municipalities can be chosen
        q_t[v_t] += 1  # allocate an inspector to the chosen municipality
        r_t = z_i[v_t] - r_i[v_t]  # antecipated reduction for the consumer complaint rate of the chosen municipality
        z_i[v_t] = r_i[v_t]  #  new value for for the consumer complaint rate of the chosen municipality
        mask_w = W[:, v_t] > 0  # mask for municipalities influenced by the chosen municipality
        y_i[mask_w] = y_i[mask_w] - W[mask_w, v_t]*r_t  # new regulator's expectations of municipalities influenced by the chosen municipality
        mask_y = y_i < 1  # mask for the lower bound of regulator's expectations
        y_i[mask_y] = 1  # new relevant regulator's expectations of municipalities influenced by the chosen municipality
        d_i[mask_w] = z_i[mask_w]/y_i[mask_w]  # new relevant discrepancies of municipalities influenced by the chosen municipality
        d_i[v_t] = z_i[v_t]/y_i[v_t]  # new relevant discrepancies of the chosen municipality
    q_t = q_t[tau]  # dropping municipalities with no allocated inspectors
    return tau, q_t  # municipalities chosen and quantity of inspectors allocated

def sps_x_dense_vecs(sps_mat, dense_vecs):
    '''
    Multiply a array of dense vectors by a sparse matrix

    Parameters
    ----------
    sps_mat : Block Sparse Row matrix
        Sparse matrix version of the adjacency matrix W.
    dense_vecs : (vertices, rounds, realizations) ndarray
        ndarray with all consumer complaint rates.
    
    Returns
    -------
    out : (vertices, rounds, realizations) ndarray
        regulator's expectations for all consumer complaint rates.
        
    '''
    rows, cols = sps_mat.shape
    _, samples, runs = dense_vecs.shape
    dense_vecs = dense_vecs.reshape(cols, -1)
    out = sps_mat.dot(dense_vecs)
    return out.reshape(rows, samples, runs).astype(np.float32)

def run_inspection_rounds(method, resp, changes):
    '''
    This function runs chosen PRInT method for all inspection rounds,
    with or without changes in consumer complaint rates, and
    simulates the responsiveness of the method

    Parameters
    ----------
    method : string
        name of the method chosen (randomness, rate ou proposed)
    resp : boll
        whether the responsiveness of the method should be calculated
    changes : boll
        whether the changes in consumer complaint rates should be applied
    
    Returns
    -------
    x_i : (vertices, rounds) ndarray
        all consumer complaint rates for one realization.
        
    '''
    global vertices, rounds, u0, eta, M, delta_ch, s, c, realization, vi_ch
    x_i = np.zeros([vertices, rounds], dtype = np.float32)  # array for consumer complaint rates for all inspection rounds
    U_i = u0.copy()  # array for average consumer complaint rates for all inspection rounds
    for n in range(rounds):  # for each inspection round n do
        if changes:
            if n == n_ch:
                U_i[vi_ch[:,realization]] = U_i[vi_ch[:,realization]] + delta_ch
                changes = False
        x_i[:, n] = U_i + eta[:, n]  # new consumer complaint rate
        mask_x = x_i[:, n] < 0  # mask to limit consumer complaint rate to non-negative values
        x_i[mask_x, n] = 0  # limiting consumer complaint rate
        if method == 'randomness':
            Tau = np.random.randint(0, vertices, [M, rounds])  # randomly chosen municipalities for all inspection rounds
            tau, q_t = PRInT(Tau[:, n])  # municipalities chosen and quantity of inspectors allocated by randomness method
        elif method == 'rate':
            tau, q_t = PRInT_rt(x_i[:, n])  # municipalities chosen and quantities of inspectors allocated by rate method
            if resp:
                x_min[n, realization] = x_i[tau[-1], n]  # lower consumer complaint rate that allocate an inspector to a new municipality in this round
        elif method == 'proposed':
            f = np.exp(-beta_x/x_i[:, n].mean().clip(min=1e-6))  # value for function f (line 9 of algorithm 1)
            tau, q_t = PRInT_pr(x_i[:, n], f, n, resp)  # municipalities chosen and quantities of inspectors allocated by proposed method
        else:
            print('The chosen method was not defined.')
            return None
        s_t = s[:tau.size, n, :]/np.tile(c[tau].reshape((-1, 1)), (1, R))  # R random capacities of inspectors for each chosen municipality
        t_t = np.zeros(tau.size, dtype = np.float32)  # array for teams' capacities
        for i_t, q_i in enumerate(q_t.astype(int)):
            t_t[i_t] = s_t[i_t, :q_i].sum()  # resultants teams' capacities
        U_i[tau] = U_i[tau] - t_t  # reducing the average consumer complaint rate
        mask_u = U_i < 0  # mask to limit average consumer complaint rate to non-negative values
        U_i[mask_u] = 0  # limiting average consumer complaint rate to non-negative values
    return x_i

def avg(x):
    '''
    This function calculates the average consumer complaint rate for every inspection round.

    Parameters
    ----------
    x : (vertices, rounds, realizations) ndarray
        all consumer complaint rates of municipalities
    
    Returns
    -------
    mu_eta : (rounds, ) ndarray
        average consumer complaint rate per inspection round
        
    '''
    return x.mean(axis=0).mean(axis=1, dtype = np.float32)

def troubled(x):
    '''
    This function calculates the average number of troubled municipalities for every inspection round.

    Parameters
    ----------
    x : (vertices, rounds, realizations) ndarray
        all consumer complaint rates of municipalities
    
    Returns
    -------
    tr : (rounds, ) ndarray
        average number of troubled municipalities per inspection round
        
    '''
    tr = x > 1
    return tr.sum(axis=0).mean(axis=1, dtype = np.float32)

def relevant_d(x):
    '''
    This function calculates the relevant discrepancy of municipalities.

    Parameters
    ----------
    x : (vertices, rounds, realizations) ndarray
        all consumer complaint rates of municipalities
    
    Returns
    -------
    d : (vertices, rounds, realizations) ndarray
        all relevant discrepancy of municipalities
        
    '''
    global Ws
    Y = sps_x_dense_vecs(Ws, x)  # regulator's expectations based on the influences between municipalitites
    mask = Y < 1  # mask for irrelevant
    Y[mask] = 1  # relevant regulator's expectations
    return x/Y  # relevant discrepancies

def max_d(d):
    '''
    This function calculates the average maximum relevant discrepancy of municipalities.

    Parameters
    ----------
    d : (vertices, rounds, realizations) ndarray
        all relevant discrepancy of municipalities
    
    Returns
    -------
    max_d : (rounds, ) ndarray
        average maximum relevant discrepancy of municipalities per inspection rounds
        
    '''
    return np.mean(d.max(axis=0), axis=1, dtype = np.float32)

def sum_d(x, d):
    '''
    This function calculates the average sum of relevant discrepancies of troubled municipalities.

    Parameters
    ----------
    d : (vertices, rounds, realizations) ndarray
        all relevant discrepancy of municipalities
    
    Returns
    -------
    t_d : (rounds, ) ndarray
        average sum of relevant discrepancies of troubled municipalities
        
    '''
    mask = x > 1
    t_d = np.where(mask,d,0)
    return t_d.sum(axis=0).mean(axis=1, dtype = np.float32)

def evaluation_metrics(x):
    '''
    This function calculates the average evaluation metrics per inspection round across municipalities and realizations.

    Parameters
    ----------
    x : (vertices, rounds, realizations) ndarray
        all consumer complaint rates of municipalities
    
    Returns
    -------
    metrics : [(rounds, ), (rounds, ), (rounds, ), (rounds, )] ndarray
        average evaluation metrics per inspection round across municipalities and realizations.
        
    '''
    d = relevant_d(x)  # relevant discrepancies
    max_td = max_d(d)  # maximum relevant discrepancy of municipalities
    sum_td = sum_d(x, d)  # sum of relevant discrepancies of troubled municipalities
    avg_x = avg(x)  # average of consumer complaint rates
    tro_v = troubled(x)  # average of number of troubled municipalities
    return max_td, sum_td, avg_x, tro_v

def resp_of_rt(x, delta, th):
    '''
    This function flags the municipalities where changes would be detected immediately by rate method.

    Parameters
    ----------
    x : (vertices, rounds, realizations) ndarray
        all consumer complaint rates of municipalities
    delta : (3, ) ndarray
        increases in consumer complaint rate
    th : (rounds, realizations) ndarray
        threshold of detection to allocate an inspector to a not yet chosen municipality
    
    Returns
    -------
    ch_rt : (vertices, rounds, realizations) ndarray
        municipalities where changes would be detected immediately by rate method
        
    '''
    global vertices, rounds, realizations
    ch_rt = np.zeros([delta.size, vertices, rounds, realizations])
    for i, inc in enumerate(delta):
        ch_rt[i,:,:,:] = x + inc > th
    return ch_rt

def responsiveness(detected):
    '''
    This function calculates the average percentages of municipalities where changes would be detected immediately.

    Parameters
    ----------
    x : (vertices, rounds, realizations) ndarray
        all consumer complaint rates of municipalities
    
    Returns
    -------
    per : (vertices, rounds, realizations) ndarray
        average percentages of municipalities per inspection round where changes would be detected immediately
        
    '''
    global vertices
    return (100/vertices)*(detected.sum(axis=0).mean(axis=1))

def mri(met_pr, met_rt):
    '''
    This function calculates the maximum relative improvement on reduction of an evaluation metric due to 
    the use of the proposed method when compared to the rate method, and the inspection round when it occurs.

    Parameters
    ----------
    met_pr : (rounds, ) ndarray
        evaluation metric of proposed method
    met_rt : (rounds, ) ndarray
        evaluation metric of rate method
    
    Returns
    -------
    mri : (rounds, ) ndarray
        maximum relative improvement
    n : int
        inspection round when it ocurrs
        
    '''
    red_pr = met_pr[0] - met_pr
    red_rt = met_rt[0] - met_rt
    imp = 100*(red_pr-red_rt)/red_rt
    
    return np.rint(np.nanmax(imp)).astype(int), np.nanargmax(imp)

def mri_ceil(met_pr, met_rt):
    '''
    This function calculates the maximum relative improvement on reduction of an evaluation metric due to 
    the use of the proposed method when compared to the rate method, and the inspection round when it occurs.
    Observe that this function ceils the reduction in evaluation metric.

    Parameters
    ----------
    met_pr : (rounds, ) ndarray
        evaluation metric of proposed method
    met_rt : (rounds, ) ndarray
        evaluation metric of rate method
    
    Returns
    -------
    mri : (rounds, ) ndarray
        maximum relative improvement
    n : int
        inspection round when it ocurrs
        
    '''
    red_pr = np.ceil(met_pr[0]).clip(min=0) - np.ceil(met_pr).clip(min=0)
    red_rt = np.ceil(met_rt[0]).clip(min=0) - np.ceil(met_rt).clip(min=0)
    imp = np.zeros_like(red_pr)
    np.divide(red_pr-red_rt, red_rt, out = imp, where = red_rt > 0, dtype=np.float32)
    imp = 100*imp
    
    return np.rint(np.nanmax(imp)).astype(int), np.nanargmax(imp)

def changed_data(x):
    '''
    This function selects data just from municipalities where changes occur.

    Parameters
    ----------
    x : (vertices, rounds, realizations) ndarray
        all consumer complaint rates of municipalities
    
    Returns
    -------
    x_p : (N_ch, rounds, realizations) ndarray
        data just from municipalities where changes occur
        
    '''
    global N_ch, vi_ch, rounds, realizations
    x_p = np.zeros([N_ch, rounds, realizations], dtype=np.float32)
    for realization in np.arange(realizations):
        x_p[:,:,realization] = x[vi_ch[:,realization],:,realization]
    return x_p

def diff_changed_evaluation_metrics(x_ch, x_no):
    '''
    This function calculates the differences on evaluation metrics just of municipalities where changes occur.

    Parameters
    ----------
    x_ch : (vertices, rounds, realizations) ndarray
        consumer complaint rates of all municipalities when changes occur
    x_no : (vertices, rounds, realizations) ndarray
        consumer complaint rates of all municipalities when changes do not occur
    
    Returns
    -------
    metrics : [(rounds, ), (rounds, ), (rounds, ), (rounds, )] ndarray
        average evaluation metrics per inspection round just of municipalities where changes occur
        
    '''    
    # relevant discrepancies
    d_ch = relevant_d(x_ch)
    d_no = relevant_d(x_no)
    
    # selecting data just from municipalities where changes occur
    d_ch_p = changed_data(d_ch)
    d_no_p = changed_data(d_no)
    x_ch_p = changed_data(x_ch)
    x_no_p = changed_data(x_no)
    tr_ch_p = x_ch_p > 1
    tr_no_p = x_no_p > 1
    
    
    max_d_ch_p = np.max(d_ch_p, axis=0)
    max_d_no_p = np.max(d_no_p, axis=0)
    sum_d_ch_p = np.sum(d_ch_p, axis=0)
    sum_d_no_p = np.sum(d_no_p, axis=0)
    avg_x_ch_p = np.mean(x_ch_p, axis=0)
    avg_x_no_p = np.mean(x_no_p, axis=0)
    tro_v_ch_p = np.sum(tr_ch_p, axis=0)
    tro_v_no_p = np.sum(tr_no_p, axis=0)
    
    # calculating the differences on data when changes occur and when they do not 
    diff_max_d_p = max_d_ch_p - max_d_no_p
    diff_sum_d_p = sum_d_ch_p - sum_d_no_p
    diff_avg_x_p = avg_x_ch_p - avg_x_no_p
    diff_tro_v_p = tro_v_ch_p - tro_v_no_p
    
    # returning the average differences
    return diff_max_d_p.mean(axis=1), diff_sum_d_p.mean(axis=1), diff_avg_x_p.mean(axis=1), diff_tro_v_p.mean(axis=1)

def progressBar(iteration, total, t0, prefix = '', suffix = '', decimals = 1, length = 100, fill = '█', printEnd = '\r'):
    '''
    This function creates a terminal progress bar.

    Parameters
    ----------
    iteration : int
        currnt iteration
    total : int
        total iterations
    prefix : string
        prefix string
    suffix : string
        suffix string
    decimals : int
        positive number of decimals in percent complete
    length : int
        character length of bar
    fill : string
        bar fill character
    printEnd : string
        end character (e.g. "\r", "\r\n")
    t0 : float
        initial time
    '''
    percent = ("{0:." + str(decimals) + "f}").format(100 * (iteration / float(total)))
    filledLength = int(length * iteration // total)
    bar = fill * filledLength + '-' * (length - filledLength)
    t1 = time.time()
    eta = int((total-iteration)*(t1 - t0)/(60*iteration))
    print(f'\r{prefix}: |{bar}| {percent}% {suffix}. ETA: {str(eta)} min.', end = printEnd)
    # Print New Line on Complete
    if iteration == total: 
        print(f'\r{prefix}: |{bar}| {percent}% {suffix}. ETA: 0 min. Total time: {int((time.time() - t0)/1)} min.')

In [ ]:
# IF YOU NEED JUST VIEW THE RESULTS, COMMENT THIS CELL (Select all and "Ctrl + /") OR CONVERT IT ON A RAW CELL

# Simulation

# loading data
data = np.load('data_for_print_simulations.npz')
W = data['W'].astype(np.float32)  # influences between municipalities
u0 = data['u0'].astype(np.float32)  # average consumer complaint rate by municipality of Brazil in July 2022
c = data['c'].astype(np.float32)  # consumers/50, 000 by municipality of Brazil in July 2022

# setting variables
vertices = W.shape[0]  # number of municipalilites of Brazil
insp_cons = np.ceil(c).astype(int)  # inspectors to be allocated for municipalities based on their number of consumers
Ws = bsr_matrix(W) # converting W in sparse matrix for faster multiplication
vi_ch = np.random.randint(0,vertices-1,[N_ch,realizations])  # N_ch municipalities where the change took place (for Figures 6 and 7)

for mu in Mu:
    # setting variables dependents of mu
    mu_i = mu/c  # average capacity of one inspector at the municipalities
    alpha = 2*mu/3 # used to set the capacity of one inspector at a municipality
    beta_x = 1 + mu**np.sign(mu - 1) # used for function f (line 9 of algorithm 1)

    # arrays for consumer complaint rates for all inspection rounds and realizations
    x_rd = np.zeros([vertices, rounds, realizations], dtype = np.float32)  # randomness method
    x_rt = np.zeros([vertices, rounds, realizations], dtype = np.float32)  # rate method
    x_pr = np.zeros([vertices, rounds, realizations], dtype = np.float32)  # proposed method

    # initializing arrays for responsiveness
    # lower consumer complaint rate that allocate an inspector to a new municipality in rate method
    x_min = np.zeros([rounds, realizations], dtype=np.float32)  
    # lower relevant discrepancy that allocate an inspector to a new municipality in proposed method
    ch_pr_a = np.zeros([vertices, rounds, realizations], dtype=np.int8)
    ch_pr_b = np.zeros([vertices, rounds, realizations], dtype=np.int8)
    ch_pr_c = np.zeros([vertices, rounds, realizations], dtype=np.int8)

    # initializing arrays for changes
    if mu == mu_ch:
        x_rt_ch = np.zeros([vertices,rounds,realizations], dtype = np.float32) # rate method
        x_pr_ch = np.zeros([vertices,rounds,realizations], dtype = np.float32) # proposed method
    
    # realizations
    t0 = time.time()
    for realization in range(realizations):
        # random elements needed for all methods
        eta = np.random.normal(mu_eta, sigma_eta, [vertices, rounds]).astype(np.float32)  # noise for consumer complaint rates
        unif = np.random.uniform(low=0, high=1, size=[M*rounds*R, 3]).astype(np.float32)  # uniform distributions for irwin-hall distribution
        s = (alpha*unif.sum(axis=1)).reshape([M, rounds, R])  # capacities sampled for an inspector at the standard municipality

        # run methods
        x_rd[:, :, realization] = run_inspection_rounds(method='randomness', resp=False, changes=False) # randomness method
        x_rt[:, :, realization] = run_inspection_rounds(method='rate', resp=True, changes=False) # rate method
        x_pr[:, :, realization] = run_inspection_rounds(method='proposed', resp=True, changes=False) # proposed method
        if mu == mu_ch: # simulate changes for Figures 6 and 7
            x_rt_ch[:, :, realization] = run_inspection_rounds(method='rate', resp=False, changes=True) # rate method
            x_pr_ch[:, :, realization] = run_inspection_rounds(method='proposed', resp=False, changes=True) # proposed method
            
        # progress bar
        progressBar(realization+1, realizations, t0, prefix = '\mu = ' + str(mu) + ' -> Progress', suffix = 'Complete', length = 50)
            
    unif = None
    eta = None
    x_i  = None
    
    # calculating evalution metrics
    max_d_rd, sum_d_rd, avg_x_rd, tro_v_rd = evaluation_metrics(x_rd)  # randomness method
    x_rd = None
    max_d_rt, sum_d_rt, avg_x_rt, tro_v_rt = evaluation_metrics(x_rt)  # rate method
    max_d_pr, sum_d_pr, avg_x_pr, tro_v_pr = evaluation_metrics(x_pr)  # proposed method
    
    # calculating methods' responsiveness for three delta
    ch_rt_a, ch_rt_b, ch_rt_c = resp_of_rt(x_rt, delta, x_min)  # flag municipalities where changes would be detected immediately by rate method
    # municipalities where changes would be detected immediately by proposed method are already flagged
    resp_rt_a = responsiveness(ch_rt_a)
    resp_rt_b = responsiveness(ch_rt_b)
    resp_rt_c = responsiveness(ch_rt_c)
    ch_rt_a, ch_rt_b, ch_rt_c = None, None, None
    resp_pr_a = responsiveness(ch_pr_a)
    resp_pr_b = responsiveness(ch_pr_b)
    resp_pr_c = responsiveness(ch_pr_c)
    ch_pr_a, ch_pr_b, ch_pr_c = None, None, None
    
    # calculating maximum relative improvements of evaluation metrics
    np.seterr(divide='ignore', invalid='ignore') # do not show "RuntimeWarning: invalid value encountered in true_divide". 
    mri_max, ind_max = mri(max_d_pr, max_d_rt)
    mri_sum, ind_sum = mri(sum_d_pr, sum_d_rt)
    mri_avg_x, ind_avg_x = mri(avg_x_pr, avg_x_rt)
    mri_tro_v, ind_tro_v = mri_ceil(tro_v_pr, tro_v_rt)  # The reduction of troubled municipalities is rounded to the nearest integer to avoid that average reductions close to zero, from the rate method, result in extra-large numbers for relative improvement.
    
    if mu == mu_ch: # when changes must be checked        
        # calculating differences on evaluation metrics when changes occur and when they do not 
        diff_max_d_rt, diff_sum_d_rt, diff_avg_x_rt, diff_tro_v_rt = diff_changed_evaluation_metrics(x_rt_ch, x_rt)  # rate method
        diff_max_d_pr, diff_sum_d_pr, diff_avg_x_pr, diff_tro_v_pr = diff_changed_evaluation_metrics(x_pr_ch, x_pr)  # proposed method
        x_rt = x_pr = x_rt_ch = x_pr_ch = None
              
        # saving all results
        np.savez_compressed('PrinT_realizations_' + str(realizations) + '_rounds_' + str(rounds) + '_M_' + str(M) + '_R_' + str(R) + '_mu_eta_' + str(mu_eta) +'_sigma_eta_' + str(sigma_eta) + '_mu_' + str(mu) + '.npz', max_d_rd = max_d_rd, sum_d_rd = sum_d_rd, avg_x_rd = avg_x_rd, tro_v_rd = tro_v_rd, max_d_rt = max_d_rt, sum_d_rt = sum_d_rt, avg_x_rt = avg_x_rt, tro_v_rt = tro_v_rt, max_d_pr = max_d_pr, sum_d_pr = sum_d_pr, avg_x_pr = avg_x_pr, tro_v_pr = tro_v_pr, resp_rt_a = resp_rt_a, resp_rt_b = resp_rt_b, resp_rt_c = resp_rt_c, resp_pr_a = resp_pr_a, resp_pr_b = resp_pr_b, resp_pr_c = resp_pr_c, mri_max = mri_max, ind_max = ind_max, mri_sum = mri_sum, ind_sum = ind_sum, mri_avg_x = mri_avg_x, ind_avg_x = ind_avg_x, mri_tro_v = mri_tro_v, ind_tro_v = ind_tro_v, diff_max_d_rt = diff_max_d_rt, diff_max_d_pr = diff_max_d_pr, diff_sum_d_rt = diff_sum_d_rt, diff_sum_d_pr = diff_sum_d_pr, diff_avg_x_rt = diff_avg_x_rt, diff_avg_x_pr = diff_avg_x_pr, diff_tro_v_rt = diff_tro_v_rt, diff_tro_v_pr = diff_tro_v_pr)
        max_d_rd = sum_d_rd = avg_x_rd = tro_v_rd = max_d_rt = sum_d_rt = avg_x_rt = tro_v_rt = max_d_pr = sum_d_pr = avg_x_pr = tro_v_pr = resp_rt_a = resp_rt_b = resp_rt_c = resp_pr_a = resp_pr_b = resp_pr_c = mri_max = ind_max = mri_sum = ind_sum = mri_avg_x = ind_avg_x = mri_tro_v = ind_tro_v = diff_max_d_rt = diff_max_d_pr = diff_sum_d_rt = diff_sum_d_pr = diff_avg_x_rt = diff_avg_x_pr = diff_tro_v_rt = diff_tro_v_pr = None
    else:
        x_rt = x_pr = None
        np.savez_compressed('PrinT_realizations_' + str(realizations) + '_rounds_' + str(rounds) + '_M_' + str(M) + '_R_' + str(R) + '_mu_eta_' + str(mu_eta) +'_sigma_eta_' + str(sigma_eta) + '_mu_' + str(mu) + '.npz', max_d_rd = max_d_rd, sum_d_rd = sum_d_rd, avg_x_rd = avg_x_rd, tro_v_rd = tro_v_rd, max_d_rt = max_d_rt, sum_d_rt = sum_d_rt, avg_x_rt = avg_x_rt, tro_v_rt = tro_v_rt, max_d_pr = max_d_pr, sum_d_pr = sum_d_pr, avg_x_pr = avg_x_pr, tro_v_pr = tro_v_pr, resp_rt_a = resp_rt_a, resp_rt_b = resp_rt_b, resp_rt_c = resp_rt_c, resp_pr_a = resp_pr_a, resp_pr_b = resp_pr_b, resp_pr_c = resp_pr_c, mri_max = mri_max, ind_max = ind_max, mri_sum = mri_sum, ind_sum = ind_sum, mri_avg_x = mri_avg_x, ind_avg_x = ind_avg_x, mri_tro_v = mri_tro_v, ind_tro_v = ind_tro_v)
        max_d_rd = sum_d_rd = avg_x_rd = tro_v_rd = max_d_rt = sum_d_rt = avg_x_rt = tro_v_rt = max_d_pr = sum_d_pr = avg_x_pr = tro_v_pr = resp_rt_a = resp_rt_b = resp_rt_c = resp_pr_a = resp_pr_b = resp_pr_c = mri_max = ind_max = mri_sum = ind_sum = mri_avg_x = ind_avg_x = mri_tro_v = ind_tro_v = None

In [ ]:
# loading data files
dados_A = np.load('PrinT_realizations_' + str(realizations) + '_rounds_' + str(rounds) + '_M_' + str(M) + '_R_' + str(R) + '_mu_eta_' + str(mu_eta) +'_sigma_eta_' + str(sigma_eta) + '_mu_' + str(Mu[0]) + '.npz')
dados_B = np.load('PrinT_realizations_' + str(realizations) + '_rounds_' + str(rounds) + '_M_' + str(M) + '_R_' + str(R) + '_mu_eta_' + str(mu_eta) +'_sigma_eta_' + str(sigma_eta) + '_mu_' + str(Mu[1]) + '.npz')
dados_C = np.load('PrinT_realizations_' + str(realizations) + '_rounds_' + str(rounds) + '_M_' + str(M) + '_R_' + str(R) + '_mu_eta_' + str(mu_eta) +'_sigma_eta_' + str(sigma_eta) + '_mu_' + str(Mu[2]) + '.npz')

In [ ]:
# plotting Figure 2
# maximum relevant discrepancy of troubled municipalities
# SET THE VALUES FOR AXIS, ELLIPSES AND ANNOTATION PROPERLY

# loading variables
max_d_rd_A = dados_A['max_d_rd']
max_d_rt_A = dados_A['max_d_rt']
max_d_pr_A = dados_A['max_d_pr']
max_d_rd_B = dados_B['max_d_rd']
max_d_rt_B = dados_B['max_d_rt']
max_d_pr_B = dados_B['max_d_pr']
max_d_rd_C = dados_C['max_d_rd']
max_d_rt_C = dados_C['max_d_rt']
max_d_pr_C = dados_C['max_d_pr']

sc = 5
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(3*sc,1*sc), constrained_layout=True, dpi=120)
ax1.plot(max_d_rd_B,'gray',label='randomness')
ax1.plot(max_d_rt_B,'darkorchid',label='rate')
ax1.plot(max_d_pr_B,'darkcyan',label='proposed')
ax1.axis([0,max_d_rd_B.size-1,0,ax1.axis()[3]])
ax1.legend(fontsize=15)
ax1.set_title('(a)', fontsize=15)
ax1.set_xlabel('Inspection round', fontsize=15)
ax1.set_ylabel('Maximum relevant discrepancy\nof troubled municipalities', fontsize=15)
ax2.plot(max_d_rt_C,'purple',label='_rate')
ax2.plot(max_d_rt_B,'darkorchid',label='rate')
ax2.plot(max_d_rt_A,'mediumorchid',label='_rate')
ax2.plot(max_d_pr_C,'darkslategray',label='_proposed')
ax2.plot(max_d_pr_B,'darkcyan',label='proposed')
ax2.plot(max_d_pr_A,'turquoise',label='_proposed')
ax2.axis([0,max_d_rt_C.size-1,1,9.2])
ell5 = Ellipse(xy=(175,2), width=70, height=0.75, angle=0, linestyle='--', fill=False)
ax2.add_artist(ell5)
ell2 = Ellipse(xy=(725,2.3), width=70, height=0.5, angle=0, linestyle='--', fill=False)
ax2.add_artist(ell2)
ell1 = Ellipse(xy=(1430,2.5), width=70, height=0.75, angle=0, linestyle='--', fill=False)
ax2.add_artist(ell1)
x_int5 = 195
y_int5 = 1.6
x_int2 = 760
y_int2 = 2.3
x_int1 = 1430
y_int1 = 2.9
ax2.annotate(
    '$\mu = ' + str(Mu[0]) + '$', fontsize=15,
    xy=(x_int5+1, y_int5), xycoords='data',
    xytext=(30, -8), textcoords='offset points',
    arrowprops=dict(arrowstyle="->"))
ax2.annotate(
    '$\mu = ' + str(Mu[1]) + '$', fontsize=15,
    xy=(x_int2+1, y_int2), xycoords='data',
    xytext=(35, 20), textcoords='offset points',
    arrowprops=dict(arrowstyle="->"))
ax2.annotate(
    '$\mu = ' + str(Mu[2]) + '$', fontsize=15,
    xy=(x_int1+1, y_int1), xycoords='data',
    xytext=(-7, 30), textcoords='offset points',
    arrowprops=dict(arrowstyle="->"))
ax2.legend(fontsize=15, loc='upper right')
ax2.set_title('(b)', fontsize=15)
ax2.set_xlabel('Inspection round', fontsize=15)
ax2.set_ylabel('Maximum relevant discrepancy\nof troubled municipalities', fontsize=15)
plt.savefig('Figure_2.png', format='png', dpi=120)
plt.show()

In [ ]:
# plotting Figure 3
# sum of relevant discrepancy of troubled municipalities
# SET THE VALUES FOR AXIS, ELLIPSES AND ANNOTATION PROPERLY

sum_d_rd_A = dados_A['sum_d_rd']
sum_d_rt_A = dados_A['sum_d_rt']
sum_d_pr_A = dados_A['sum_d_pr']
sum_d_rd_B = dados_B['sum_d_rd']
sum_d_rt_B = dados_B['sum_d_rt']
sum_d_pr_B = dados_B['sum_d_pr']
sum_d_rd_C = dados_C['sum_d_rd']
sum_d_rt_C = dados_C['sum_d_rt']
sum_d_pr_C = dados_C['sum_d_pr']

sc=5
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(3*sc,1*sc), constrained_layout=True, dpi=120)
ax1.plot(sum_d_rd_B,'gray',label='randomness')
ax1.plot(sum_d_rt_B,'darkorchid',label='rate')
ax1.plot(sum_d_pr_B,'darkcyan',label='proposed')
ax1.axis([0,sum_d_rd_B.size-1,120,ax1.axis()[3]])
ax1.legend(fontsize=15, loc='upper right')
ax1.set_title('(a)', fontsize=15)
ax1.set_xlabel('Inspection round', fontsize=15)
ax1.set_ylabel('Sum of relevant discrepancies\nof troubled municipalities', fontsize=15)
ax2.plot(sum_d_rt_C,'purple',label='_rate')
ax2.plot(sum_d_rt_B,'darkorchid',label='rate')
ax2.plot(sum_d_rt_A,'mediumorchid',label='_rate')
ax2.plot(sum_d_pr_C,'darkslategray',label='_proposed')
ax2.plot(sum_d_pr_B,'darkcyan',label='proposed')
ax2.plot(sum_d_pr_A,'turquoise',label='_proposed')
ax2.axis([0,sum_d_rt_C.size-1,120,3200])
ell5 = Ellipse(xy=(140,900), width=100, height=200, angle=0, linestyle='--', fill=False)
ax2.add_artist(ell5)
ell2 = Ellipse(xy=(695,900), width=100, height=200, angle=0, linestyle='--', fill=False)
ax2.add_artist(ell2)
ell1 = Ellipse(xy=(1400,900), width=100, height=200, angle=0, linestyle='--', fill=False)
ax2.add_artist(ell1)
x_int5 = 190
y_int5 = 900
x_int2 = 745
y_int2 = 900
x_int1 = 1350
y_int1 = 900
ax2.annotate(
    '$\mu = ' + str(Mu[0]) + '$', fontsize=15,
    xy=(x_int5, y_int5), xycoords='data',
    xytext=(20, -30), textcoords='offset points',
    arrowprops=dict(arrowstyle="->"))
ax2.annotate(
    '$\mu = ' + str(Mu[1]) + '$', fontsize=15,
    xy=(x_int2, y_int2), xycoords='data',
    xytext=(20, -30), textcoords='offset points',
    arrowprops=dict(arrowstyle="->"))
ax2.annotate(
    '$\mu = ' + str(Mu[2]) + '$', fontsize=15,
    xy=(x_int1, y_int1), xycoords='data',
    xytext=(-70, -30), textcoords='offset points',
    arrowprops=dict(arrowstyle="->"))
ax2.legend(fontsize=15, loc='upper right')
ax2.set_title('(b)', fontsize=15)
ax2.set_xlabel('Inspection round', fontsize=15)
ax2.set_ylabel('Sum of relevant discrepancies\nof troubled municipalities', fontsize=15)
plt.savefig('Figure_3.png', format='png', dpi=120)
plt.show()

In [ ]:
# plotting Figure 4
# average consumer complaint rates and number of troubled municipalities
# SET THE VALUES FOR AXIS, ELLIPSES AND ANNOTATION PROPERLY

avg_x_rd_A = dados_A['avg_x_rd']
avg_x_rt_A = dados_A['avg_x_rt']
avg_x_pr_A = dados_A['avg_x_pr']
avg_x_rd_B = dados_B['avg_x_rd']
avg_x_rt_B = dados_B['avg_x_rt']
avg_x_pr_B = dados_B['avg_x_pr']
avg_x_rd_C = dados_C['avg_x_rd']
avg_x_rt_C = dados_C['avg_x_rt']
avg_x_pr_C = dados_C['avg_x_pr']
tro_v_rd_A = dados_A['tro_v_rd']
tro_v_rt_A = dados_A['tro_v_rt']
tro_v_pr_A = dados_A['tro_v_pr']
tro_v_rd_B = dados_B['tro_v_rd']
tro_v_rt_B = dados_B['tro_v_rt']
tro_v_pr_B = dados_B['tro_v_pr']
tro_v_rd_C = dados_C['tro_v_rd']
tro_v_rt_C = dados_C['tro_v_rt']
tro_v_pr_C = dados_C['tro_v_pr']

sc=5
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(3*sc,1*sc), constrained_layout=True, dpi=120)
ax1.plot(avg_x_rt_C,'purple',label='_rate')
ax1.plot(avg_x_pr_C,'darkslategray',label='_proposed')
ax1.plot(avg_x_rt_B,'darkorchid',label='rate')
ax1.plot(avg_x_pr_B,'darkcyan',label='proposed')
ax1.plot(avg_x_rt_A,'mediumorchid',label='_rate')
ax1.plot(avg_x_pr_A,'turquoise',label='_proposed')
ax1.axis([0,avg_x_rt_C.size-1,0,16.5])
ell5 = Ellipse(xy=(90,4), width=90, height=1.5, angle=0, linestyle='--', fill=False)
ax1.add_artist(ell5)
ell2 = Ellipse(xy=(470,4), width=230, height=1.5, angle=0, linestyle='--', fill=False)
ax1.add_artist(ell2)
ell1 = Ellipse(xy=(1050,4), width=280, height=1.5, angle=0, linestyle='--', fill=False)
ax1.add_artist(ell1)
x_int5 = 135
y_int5 = 3.8
x_int2 = 580
y_int2 = 3.8
x_int1 = 1185
y_int1 = 3.8
ax1.annotate(
    '$\mu = ' + str(Mu[0]) + '$', fontsize=15,
    xy=(x_int5, y_int5), xycoords='data',
    xytext=(20, -40), textcoords='offset points',
    arrowprops=dict(arrowstyle="->"))
ax1.annotate(
    '$\mu = ' + str(Mu[1]) + '$', fontsize=15,
    xy=(x_int2, y_int2), xycoords='data',
    xytext=(30, -30), textcoords='offset points',
    arrowprops=dict(arrowstyle="->"))
ax1.annotate(
    '$\mu = ' + str(Mu[2]) + '$', fontsize=15,
    xy=(x_int1, y_int1), xycoords='data',
    xytext=(30, -20), textcoords='offset points',
    arrowprops=dict(arrowstyle="->"))
ax1.legend(fontsize=15, loc='upper right')
ax1.set_title('(b)', fontsize=15)
ax1.set_xlabel('Inspection round', fontsize=15)
ax1.set_ylabel('Average complaint rate', fontsize=15)
ax2.plot(tro_v_rt_C,'purple',label='_rate')
ax2.plot(tro_v_rt_B,'darkorchid',label='rate')
ax2.plot(tro_v_rt_A,'mediumorchid',label='_nrate')
ax2.plot(tro_v_pr_C,'darkslategray',label='_proposed')
ax2.plot(tro_v_pr_B,'darkcyan',label='proposed')
ax2.plot(tro_v_pr_A,'turquoise',label='_proposed')
ax2.axis([0,tro_v_rt_C.size-1,100,3550])
ell5 = Ellipse(xy=(142,1000), width=70, height=200, angle=0, linestyle='--', fill=False)
ax2.add_artist(ell5)
ell2 = Ellipse(xy=(700,1000), width=70, height=200, angle=0, linestyle='--', fill=False)
ax2.add_artist(ell2)
ell1 = Ellipse(xy=(1400,1000), width=70, height=200, angle=0, linestyle='--', fill=False)
ax2.add_artist(ell1)
x_int5 = 180
y_int5 = 1000
x_int2 = 738
y_int2 = 1000
x_int1 = 1362
y_int1 = 1000
ax2.annotate(
    '$\mu = ' + str(Mu[0]) + '$', fontsize=15,
    xy=(x_int5, y_int5), xycoords='data',
    xytext=(20, -30), textcoords='offset points',
    arrowprops=dict(arrowstyle="->"))
ax2.annotate(
    '$\mu = ' + str(Mu[1]) + '$', fontsize=15,
    xy=(x_int2, y_int2), xycoords='data',
    xytext=(20, -30), textcoords='offset points',
    arrowprops=dict(arrowstyle="->"))
ax2.annotate(
    '$\mu = ' + str(Mu[2]) + '$', fontsize=15,
    xy=(x_int1, y_int1), xycoords='data',
    xytext=(-60, -30), textcoords='offset points',
    arrowprops=dict(arrowstyle="->"))
ax2.legend(fontsize=15, loc='upper right')
ax2.set_title('(b)', fontsize=15)
ax2.set_xlabel('Inspection round', fontsize=15)
ax2.set_ylabel('Troubled municipalities', fontsize=15)
plt.savefig('Figure_4.png', format='png', dpi=120)
plt.show()

In [ ]:
# printing Table 3

mri_max_A = dados_A['mri_max']
ind_max_A = dados_A['ind_max']
mri_sum_A = dados_A['mri_sum']
ind_sum_A = dados_A['ind_sum']
mri_avg_x_A = dados_A['mri_avg_x']
ind_avg_x_A = dados_A['ind_avg_x']
mri_tro_v_A = dados_A['mri_tro_v']
ind_tro_v_A = dados_A['ind_tro_v']
mri_max_B = dados_B['mri_max']
ind_max_B = dados_B['ind_max']
mri_sum_B = dados_B['mri_sum']
ind_sum_B = dados_B['ind_sum']
mri_avg_x_B = dados_B['mri_avg_x']
ind_avg_x_B = dados_B['ind_avg_x']
mri_tro_v_B = dados_B['mri_tro_v']
ind_tro_v_B = dados_B['ind_tro_v']
mri_max_C = dados_C['mri_max']
ind_max_C = dados_C['ind_max']
mri_sum_C = dados_C['mri_sum']
ind_sum_C = dados_C['ind_sum']
mri_avg_x_C = dados_C['mri_avg_x']
ind_avg_x_C = dados_C['ind_avg_x']
mri_tro_v_C = dados_C['mri_tro_v']
ind_tro_v_C = dados_C['ind_tro_v']

data = [[           'maximum relative improvement on reduction of', '\mu=' + str(Mu[0])                              , '\mu=' + str(Mu[1])                              , '\mu=' + str(Mu[2])                              ],
        ['maximum relevant discrepancy of troubled municipalities', str(mri_max_A) + '% (' + str(ind_max_A) + ')'    , str(mri_max_B) + '% (' + str(ind_max_B) + ')'    , str(mri_max_C) + '% (' + str(ind_max_C) + ')'    ],
        [ 'sum of relevant discrepancy of troubled municipalities', str(mri_sum_A) + '% (' + str(ind_sum_A) + ')'    , str(mri_sum_B) + '% (' + str(ind_sum_B) + ')'    , str(mri_sum_C) + '% (' + str(ind_sum_C) + ')'    ],
        [                        'average consumer complaint rate', str(mri_avg_x_A) + '% (' + str(ind_avg_x_A) + ')', str(mri_avg_x_B) + '% (' + str(ind_avg_x_B) + ')', str(mri_avg_x_C) + '% (' + str(ind_avg_x_C) + ')'],
        [                      'number of troubled municipalities', str(mri_tro_v_A) + '% (' + str(ind_tro_v_A) + ')', str(mri_tro_v_B) + '% (' + str(ind_tro_v_B) + ')', str(mri_tro_v_C) + '% (' + str(ind_tro_v_C) + ')']]

table = tabulate.tabulate(data, headers='firstrow', tablefmt='pretty')
print(table)

In [ ]:
# plotting Figure 5
# Percentage of municipalities where the change would be immediately handled by rate and proposed methods
# SET THE VALUES FOR AXIS, ELLIPSES AND ANNOTATION PROPERLY

resp_rt_a_A = dados_A['resp_rt_a']
resp_rt_b_A = dados_A['resp_rt_b']
resp_rt_c_A = dados_A['resp_rt_c']
resp_pr_a_A = dados_A['resp_pr_a']
resp_pr_b_A = dados_A['resp_pr_b']
resp_pr_c_A = dados_A['resp_pr_c']
resp_rt_a_B = dados_B['resp_rt_a']
resp_rt_b_B = dados_B['resp_rt_b']
resp_rt_c_B = dados_B['resp_rt_c']
resp_pr_a_B = dados_B['resp_pr_a']
resp_pr_b_B = dados_B['resp_pr_b']
resp_pr_c_B = dados_B['resp_pr_c']
resp_rt_a_C = dados_C['resp_rt_a']
resp_rt_b_C = dados_C['resp_rt_b']
resp_rt_c_C = dados_C['resp_rt_c']
resp_pr_a_C = dados_C['resp_pr_a']
resp_pr_b_C = dados_C['resp_pr_b']
resp_pr_c_C = dados_C['resp_pr_c']

sc=5
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(3*sc,1*sc), constrained_layout=True, dpi=120)
ax1.plot(resp_rt_a_A, 'purple', label='_rate')
ax1.plot(resp_pr_a_A, 'darkslategray', label='_proposed')
ax1.plot(resp_rt_b_A, 'darkorchid', label='rate')
ax1.plot(resp_pr_b_A, 'darkcyan', label='proposed')
ax1.plot(resp_rt_c_A, 'mediumorchid', label='_rate')
ax1.plot(resp_pr_c_A, 'darkturquoise', label='_proposed')
ax1.axis([0,185,0,102])
ell5 = Ellipse(xy=(50,16), width=6, height=12, angle=0, linestyle='--', fill=False)
ax1.add_artist(ell5)
ell2 = Ellipse(xy=(75,7), width=10, height=6, angle=0, linestyle='--', fill=False)
ax1.add_artist(ell2)
ell1 = Ellipse(xy=(174,36), width=10, height=6, angle=0, linestyle='--', fill=False)
ax1.add_artist(ell1)
x_int5 = 50
y_int5 = 23
x_int2 = 80
y_int2 = 10
x_int1 = 174
y_int1 = 40
ax1.annotate(
    '$\Delta = ' + str(delta[0]) + '$', fontsize=15,
    xy=(x_int5, y_int5), xycoords='data',
    xytext=(20, 5), textcoords='offset points',
    arrowprops=dict(arrowstyle="->"))
ax1.annotate(
    '$\Delta = ' + str(delta[1]) + '$', fontsize=15,
    xy=(x_int2, y_int2), xycoords='data',
    xytext=(20, 5), textcoords='offset points',
    arrowprops=dict(arrowstyle="->"))
ax1.annotate(
    '$\Delta = ' + str(delta[2]) + '$', fontsize=15,
    xy=(x_int1, y_int1), xycoords='data',
    xytext=(-21, 25), textcoords='offset points',
    arrowprops=dict(arrowstyle="->"))
ax1.legend(fontsize=15, loc='upper left')
ax1.set_title('(a)', fontsize=15)
ax1.set_xlabel('Inspection round', fontsize=15)
ax1.set_ylabel('Percentage of municipalities', fontsize=15)
ax2.plot(resp_rt_a_B, 'purple', label='_rate')
ax2.plot(resp_pr_a_B, 'darkslategray', label='_proposed')
ax2.plot(resp_rt_b_B, 'darkorchid', label='rate')
ax2.plot(resp_pr_b_B, 'darkcyan', label='proposed')
ax2.plot(resp_rt_c_B, 'mediumorchid', label='_rate')
ax2.plot(resp_pr_c_B, 'darkturquoise', label='_proposed')
ax2.axis([0,800,0,102])
ell5 = Ellipse(xy=(350,47), width=20, height=42, angle=0, linestyle='--', fill=False)
ax2.add_artist(ell5)
ell2 = Ellipse(xy=(400,18), width=40, height=10, angle=0, linestyle='--', fill=False)
ax2.add_artist(ell2)
ell1 = Ellipse(xy=(490,8), width=40, height=6, angle=0, linestyle='--', fill=False)
ax2.add_artist(ell1)
x_int5 = 362
y_int5 = 52
x_int2 = 420
y_int2 = 22
x_int1 = 510
y_int1 = 11
ax2.annotate(
    '$\Delta = ' + str(delta[0]) + '$', fontsize=15,
    xy=(x_int5, y_int5), xycoords='data',
    xytext=(20, 20), textcoords='offset points',
    arrowprops=dict(arrowstyle="->"))
ax2.annotate(
    '$\Delta = ' + str(delta[1]) + '$', fontsize=15,
    xy=(x_int2, y_int2), xycoords='data',
    xytext=(20, 20), textcoords='offset points',
    arrowprops=dict(arrowstyle="->"))
ax2.annotate(
    '$\Delta = ' + str(delta[2]) + '$', fontsize=15,
    xy=(x_int1, y_int1), xycoords='data',
    xytext=(20, 10), textcoords='offset points',
    arrowprops=dict(arrowstyle="->"))
ax2.legend(fontsize=15, loc='upper left')
ax2.set_title('(b)', fontsize=15)
ax2.set_xlabel('Inspection round', fontsize=15)
ax3.plot(resp_rt_a_C, 'purple', label='_rate')
ax3.plot(resp_pr_a_C, 'darkslategray', label='_proposed')
ax3.plot(resp_rt_b_C, 'darkorchid', label='rate')
ax3.plot(resp_pr_b_C, 'darkcyan', label='proposed')
ax3.plot(resp_rt_c_C, 'mediumorchid', label='_rate')
ax3.plot(resp_pr_c_C, 'darkturquoise', label='_proposed')
ax3.axis([0,1600,0,102])
ell5 = Ellipse(xy=(600,43), width=40, height=33, angle=0, linestyle='--', fill=False)
ax3.add_artist(ell5)
ell2 = Ellipse(xy=(800,22), width=80, height=8, angle=0, linestyle='--', fill=False)
ax3.add_artist(ell2)
ell1 = Ellipse(xy=(1000,10), width=80, height=8, angle=0, linestyle='--', fill=False)
ax3.add_artist(ell1)
x_int5 = 625
y_int5 = 45
x_int2 = 801
y_int2 = 27
x_int1 = 1022
y_int1 = 14
ax3.annotate(
    '$\Delta = ' + str(delta[0]) + '$', fontsize=15,
    xy=(x_int5, y_int5), xycoords='data',
    xytext=(20, 20), textcoords='offset points',
    arrowprops=dict(arrowstyle="->"))
ax3.annotate(
    '$\Delta = ' + str(delta[1]) + '$', fontsize=15,
    xy=(x_int2, y_int2), xycoords='data',
    xytext=(20, 25), textcoords='offset points',
    arrowprops=dict(arrowstyle="->"))
ax3.annotate(
    '$\Delta = ' + str(delta[2]) + '$', fontsize=15,
    xy=(x_int1, y_int1), xycoords='data',
    xytext=(20, 15), textcoords='offset points',
    arrowprops=dict(arrowstyle="->"))
ax3.legend(fontsize=15, loc='upper left')
ax3.set_title('(c)', fontsize=15)
ax3.set_xlabel('Inspection round', fontsize=15)
plt.savefig('Figure_5.png', format='png', dpi=120)
plt.show()

In [ ]:
# plotting Figure 6
# differences when changes occur and when they do not on (a) maximum relevant discrepancy of troubled municipalities, and (b) sum of relevant discrepancies of troubled municipalities
# SET THE VALUES FOR AXIS, ELLIPSES AND ANNOTATION PROPERLY

diff_max_d_rt_B = dados_B['diff_max_d_rt']
diff_max_d_pr_B = dados_B['diff_max_d_pr']
diff_sum_d_rt_B = dados_B['diff_sum_d_rt']
diff_sum_d_pr_B = dados_B['diff_sum_d_pr']
sc = 5
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(3*sc,1*sc), constrained_layout=True, dpi=120)
ax1.plot(diff_max_d_rt_B,'darkorchid',label='rate')
ax1.plot(diff_max_d_pr_B,'darkcyan',label='proposed')
ax1.axis([0,800,0,ax1.axis()[3]])
ax1.legend(fontsize=15)
ax1.set_title('(b)', fontsize=15)
ax1.set_xlabel('Inspection round', fontsize=15)
ax1.set_ylabel('Differences on maximum relevant\ndiscrepancy of troubled municipalities', fontsize=15)
ax2.plot(diff_sum_d_rt_B,'darkorchid',label='rate')
ax2.plot(diff_sum_d_pr_B,'darkcyan',label='proposed')
ax2.axis([0,800,0,ax2.axis()[3]])
ax2.legend(fontsize=15)
ax2.set_title('(a)', fontsize=15)
ax2.set_xlabel('Inspection round', fontsize=15)
ax2.set_ylabel('Differences on sum of relevant\ndiscrepancies of troubled municipalities', fontsize=15)
plt.savefig('Figure_6.png', format='png', dpi=120)
plt.show()

In [ ]:
# plotting Figure 7
# differences when changes occur and when they do not on (a) average consumer complaint rate, and (b) number of troubled municipalities
# SET THE VALUES FOR AXIS, ELLIPSES AND ANNOTATION PROPERLY

diff_avg_x_rt_B = dados_B['diff_avg_x_rt']
diff_avg_x_pr_B = dados_B['diff_avg_x_pr']
diff_tro_v_rt_B = dados_B['diff_tro_v_rt']
diff_tro_v_pr_B = dados_B['diff_tro_v_pr']

sc = 5
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(3*sc,1*sc), constrained_layout=True, dpi=120)
ax1.plot(diff_avg_x_rt_B,'darkorchid')
ax1.plot(diff_avg_x_pr_B,'darkcyan')
ax1.legend(['rate','proposed'], fontsize=15)
ax1.axis([0,800,0,ax1.axis()[3]])
ax1.set_title('(a)', fontsize=15)
ax1.set_xlabel('Inspection round', fontsize=15)
ax1.set_ylabel('Differences on average complaint rate', fontsize=15)
ax2.plot(diff_tro_v_rt_B,'darkorchid',label='rate')
ax2.plot(diff_tro_v_pr_B,'darkcyan',label='proposed')
ax2.axis([0,800,0,ax2.axis()[3]])
ax2.legend(fontsize=15)
ax2.set_title('(b)', fontsize=15)
ax2.set_xlabel('Inspection round', fontsize=15)
ax2.set_ylabel('Differences on troubled municipalities', fontsize=15)
plt.savefig('Figure_7.png', format='png', dpi=120)
plt.show()